In [8]:
import os
from dotenv import load_dotenv
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
langchain_hub_key = os.getenv("LANGCHAIN_HUB_KEY")

In [9]:
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate

In [10]:
llm = ChatOpenAI(openai_api_key=openai_api_key)

In [11]:
graph = Neo4jGraph(
    url="bolt://3.239.79.108:7687",
    username="neo4j",
    password="arrows-sewer-assistants",
)

In [12]:
CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.
Convert the user's question based on the schema.

Schema: {schema}
Question: {question}
"""

In [13]:
CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.
Convert the user's question based on the schema.

Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.

Schema: {schema}
Question: {question}
"""

In [20]:
CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.
Convert the user's question based on the schema.

Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
For movie titles that begin with "The", move "the" to the end, For example "The 39 Steps" becomes "39 Steps, The" or "The Matrix" becomes "Matrix, The".

Schema: {schema}
Question: {question}
"""

In [24]:
"""
Controlling the response
As well as instructing the LLM on how to deal with the question, you can also instruct the LLM on how to respond.

You could instruct the LLM only to respond when the Cypher statement returns data.

If no data is returned, do not attempt to answer the question.
You may want the LLM to only respond to questions in the scope of the task. For example:

Only respond to questions that require you to construct a Cypher statement.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Concise responses from the LLM may be needed:

Do not include any explanations or apologies in your responses.
Or you may want to restrict the format of the response:

Do not include any text except the generated Cypher statement.
Ultimately, you must fine-tune your instructions for the specific task to ensure the best results.

"""

'\nControlling the response\nAs well as instructing the LLM on how to deal with the question, you can also instruct the LLM on how to respond.\n\nYou could instruct the LLM only to respond when the Cypher statement returns data.\n\nIf no data is returned, do not attempt to answer the question.\nYou may want the LLM to only respond to questions in the scope of the task. For example:\n\nOnly respond to questions that require you to construct a Cypher statement.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nConcise responses from the LLM may be needed:\n\nDo not include any explanations or apologies in your responses.\nOr you may want to restrict the format of the response:\n\nDo not include any text except the generated Cypher statement.\nUltimately, you must fine-tune your instructions for the specific task to ensure the best results.\n\n'

In [21]:
cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

In [22]:
cypher_chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    cypher_prompt=cypher_generation_prompt,
    verbose=True
)

In [23]:
cypher_chain.invoke({"query": "Who acted in The Matrix and what roles did they play?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie)
WHERE m.title = 'Matrix, The'
MATCH (a:Actor)-[r:ACTED_IN]->(m)
RETURN a.name, r.role
Full Context:
[{'a.name': 'Hugo Weaving', 'r.role': 'Agent Smith'}, {'a.name': 'Laurence Fishburne', 'r.role': 'Morpheus'}, {'a.name': 'Keanu Reeves', 'r.role': 'Thomas A. Anderson / Neo'}, {'a.name': 'Carrie-Anne Moss', 'r.role': 'Trinity'}]

> Finished chain.


{'query': 'Who acted in The Matrix and what roles did they play?',
 'result': 'Hugo Weaving played Agent Smith, Laurence Fishburne played Morpheus, Keanu Reeves played Thomas A. Anderson / Neo, and Carrie-Anne Moss played Trinity in The Matrix.'}